In [1]:
# Download data, execute this block once in a while
# checks if dataset is downloaded at data/covid19-severity-prediction
# updating to the latest version
# Note: this can take a while if you don't have the dataset in data/covid19-severity-prediction

from utils import *

updating to the latest version
update successful
current working dir: d:\dataset\CS184A-Covid19_severity_prediction


In [8]:
import load_data
from Project_Models import RNN_Model
import torch
import torch.nn as nn

print("current device:", torch.cuda.get_device_name(torch.cuda.current_device()))

def normalize(x):
    """ normalize data of each column """
    maxtensor = x.max(0, keepdim=True)[0]
    maxtensor[maxtensor==0] = 1e-4
    x_normed = x / maxtensor
    return x_normed, maxtensor

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
df = load_data.load_county_level('./data/covid19-severity-prediction/data')

current device: GeForce RTX 3080
loading county-level data...
loaded and merged COVID-19 cases/deaths data successfully


In [9]:
# Preprocess Data

maxload = 100

x = form_input_tensor(df, ['#Hospitals', '#ICU_beds', 'MedicareEnrollment,AgedTot2017', 'DiabetesPercentage'], maxload=maxload).to(device)
x, xmaxtensor = normalize(x)
print(f"#x nan: {(torch.sum(torch.isnan(x)))}")
print("x:", x)

y = form_labels_tensor(df, maxload=maxload).to(device)
y, ymaxtensor = normalize(y)
print(f"#y nan: {torch.sum(torch.isnan(y))}")
print("y:", y * ymaxtensor)

Xtrain = x[:90]
Ytrain = y[:90]
Xtest  = x[90:]
Ytest  = y[90:]

#x nan: 0
x: tensor([[[0.1579, 0.1496, 0.2425, 0.7638, 0.0000, 0.0000],
         [0.1579, 0.1496, 0.2425, 0.7638, 0.0000, 0.0000],
         [0.1579, 0.1496, 0.2425, 0.7638, 0.0000, 0.0000],
         ...,
         [0.1579, 0.1496, 0.2425, 0.7638, 0.2434, 1.0000],
         [0.1579, 0.1496, 0.2425, 0.7638, 0.2433, 1.0000],
         [0.1579, 0.1496, 0.2425, 0.7638, 0.2426, 1.0000]],

        [[0.0789, 0.0607, 0.2413, 0.8031, 0.0000, 0.0000],
         [0.0789, 0.0607, 0.2413, 0.8031, 0.0000, 0.0000],
         [0.0789, 0.0607, 0.2413, 0.8031, 0.0000, 0.0000],
         ...,
         [0.0789, 0.0607, 0.2413, 0.8031, 0.2464, 0.9824],
         [0.0789, 0.0607, 0.2413, 0.8031, 0.2463, 0.9827],
         [0.0789, 0.0607, 0.2413, 0.8031, 0.2456, 0.9825]],

        [[1.0000, 1.0000, 1.0000, 0.6378, 0.0000, 0.0000],
         [1.0000, 1.0000, 1.0000, 0.6378, 0.0000, 0.0000],
         [1.0000, 1.0000, 1.0000, 0.6378, 0.0000, 0.0000],
         ...,
         [1.0000, 1.0000, 1.0000, 0.6378, 1.0000, 0.9648

In [12]:
# train model

model = RNN_Model.RNN(Xtrain.shape[2], 128, 2).to(device)
with torch.no_grad():
    print("Before training:", model(Xtrain) * ymaxtensor)

criterion = nn.MSELoss()
# criterion = nn.SmoothL1Loss()
learning_rate = 1e-3
num_epoches = 300
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epoches):
    outputs = model(Xtrain)

    loss = criterion(outputs, Ytrain)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    print(f'Epoch [{epoch + 1}/{num_epoches}], Loss: {loss.item():.4f}')
with torch.no_grad():
    print("After training:", (model(Xtest) * ymaxtensor))
    print("y:", (Ytest * ymaxtensor))

Before training: tensor([[-1.5259e+04,  6.6340e+01],
        [-1.5331e+04,  4.9460e+01],
        [-1.2547e+04,  2.6528e+02],
        [-1.2354e+04,  1.3112e+02],
        [-1.4370e+04, -4.1639e+01],
        [-1.0369e+04,  1.2539e+02],
        [-8.5304e+03,  1.3066e+02],
        [-1.0967e+04,  1.2689e+02],
        [-1.2275e+04,  1.9727e+01],
        [-9.9505e+03,  4.2874e+01],
        [-9.2117e+03,  1.0870e+02],
        [-9.6257e+03,  9.6876e+01],
        [-1.0686e+04,  2.2824e+00],
        [-9.0206e+03,  9.6022e+01],
        [-8.9036e+03,  1.0759e+02],
        [-1.0854e+04,  2.8763e+00],
        [-1.0857e+04, -4.1959e+01],
        [-9.2015e+03,  8.7703e+01],
        [-9.6140e+03,  2.2724e+01],
        [-8.8863e+03,  7.1919e+01],
        [-8.8522e+03,  4.1370e+01],
        [-9.5966e+03,  4.2470e+01],
        [-9.4445e+03,  1.1170e+02],
        [-7.8109e+03,  1.0344e+02],
        [-9.9893e+03,  6.3579e+00],
        [-8.1714e+03,  1.0065e+02],
        [-9.4291e+03,  5.5570e+01],
        [-8

In [17]:
# Saving the model
# torch.save(model, "saved_model/first100dataloss=0.model")

In [ ]:
# TODO
# 1. Batch Gradient Descent using dataloader
# 2. Handle NaN case